In [117]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [118]:
M2_df = pd.read_csv('data/M2SL.csv')

M2_df['DATE'] = pd.to_datetime(M2_df['DATE'])

M2_df.set_index('DATE', inplace=True)

M2_df['change'] = M2_df['M2SL'] -  M2_df['M2SL'].shift(1)

M2_df['^ %'] = M2_df['M2SL'].pct_change()

M2_df['Annualised Rate'] = M2_df['^ %'] * 52

In [119]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=M2_df.index,
    y=M2_df['M2SL'],
    name='M2',
    fill='tonexty',
    fillcolor='rgba(0, 0, 255, 0.9)',
    line=dict(color='blue', width=1),
))

# ---------------------------------------------------------------------

Y = np.array(M2_df['M2SL'])
X = np.arange(1, len(Y) + 1)
m, c = np.polyfit(X, Y, 1)

# ---------------------------------------------------------------------

fig.add_trace(go.Scatter(
    x=M2_df.index, 
    y=m * X + c, 
    name='Trend (LR)',
    line=dict(color='red', width=1),
    showlegend=True
))

fig.update_layout(
    title=dict(text='M2'),
    width=1400,
    height=900,
    plot_bgcolor="black",
    paper_bgcolor="#212121",
    font=dict(color="white"),
    hovermode="x unified",
    yaxis=dict(title="Billions of Dollars", exponentformat="none", showgrid=False, zeroline=False),
    barmode="group",
    legend=dict(x=0.5, y=-0.2, orientation="h"),
    xaxis=dict(showgrid=False),
)

fig.show()

In [120]:
GDP_df = pd.read_csv('data/A191RL1Q225SBEA.csv')

GDP_df['DATE'] = pd.to_datetime(GDP_df['DATE'])
GDP_df.set_index('DATE', inplace=True)
GDP_df.columns = ['GDP']

In [121]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=M2_df.index,
    y=M2_df['Annualised Rate'],
    name='M2',
    marker=dict(color='rgba(0, 255, 0, 1)'),
))

filtered_gdp_df = GDP_df[GDP_df.index >= M2_df.index[0]]

fig.add_trace(go.Bar(
    x=filtered_gdp_df.index,
    y=filtered_gdp_df['GDP'],
    name='GDP',
    marker=dict(color='rgba(255, 0, 0, 1)'),
    yaxis="y2"  
))

# ------------------------------------------------------------------------
level_a = 0  # Level on left y-axis
level_b = 0  # Matching level on right y-axis

# Define the ranges for each axis
range_left = [M2_df['Annualised Rate'].min() - .1, M2_df['Annualised Rate'].max() + .1]  # Min and max for right y-axis (to be adjusted)
range_right = [filtered_gdp_df['GDP'].min() - 5, filtered_gdp_df['GDP'].max() + 5]  # Min and max for left y-axis

# Calculate scaling factor to align levels
proportion = (level_a - range_left[0]) / (range_left[1] - range_left[0])  # Position of level_a
range_right[1] = range_right[0] + (level_b - range_right[0]) / proportion  # Adjust right range
# ------------------------------------------------------------------------

fig.update_layout(
    title=dict(text='M2 Money Supply (Annualised Rate) vs GDP change'),
    plot_bgcolor="black",
    paper_bgcolor="#212121",
    font=dict(color="white"),
    width=1400,
    height=700,
    # hovermode="y unified",
    yaxis=dict(title="M2 change %", exponentformat="none", range=range_left, showgrid=False, zeroline=False),
    yaxis2=dict(title="Real GDP in %", overlaying="y", side="right", range=range_right, showgrid=False, zeroline=False),
    barmode="group",
    legend=dict(x=0.5, y=-0.2, orientation="h"),
    xaxis=dict(showgrid=False),
)

fig.show()